In [2]:
import json
import numpy as np


camera_json_path = "/home/makramchahine/repos/gaussian-splatting/output/holodeck2/cameras.json"
with open(camera_json_path) as f:
    cameras = json.load(f)

### Helpers

In [53]:
def get_pos_rot(camera_dict):
    return np.array(camera_dict['position'].copy()), np.array(camera_dict['rotation'].copy())

def convert_to_W2C(keycamera_dict):
    origin = np.array(keycamera_dict['origin'])
    up = np.array(keycamera_dict['up'])
    target = np.array(keycamera_dict['target'])

    forward_direction = np.array([0, 0, -2.5]) - origin
    forward_direction = forward_direction / np.linalg.norm(forward_direction)

    right_direction = np.cross(up, forward_direction)
    right_direction = right_direction / np.linalg.norm(right_direction)

    up_direction = np.cross(forward_direction, right_direction)
    up_direction = up_direction / np.linalg.norm(up_direction)

    rot = np.array([up_direction, right_direction, forward_direction])

    return {
        'position': origin.tolist(),
        'rotation': rot.tolist()
    }

def parse_keycamera(file_path):
    with open(file_path, 'r') as file:
        data = file.readlines()
    parsed_data = []
    for line in data:
        line = line.strip().split('-D')
        line_dict = {}
        for item in line:
            output = item.split('=')
            if len(output) == 2:
                key, value = output
                key = key.strip()
                # if value has commas, convert it to a list
                if ',' in value:
                    value = value.split(',')
                    value = [float(v) for v in value]
                else:
                    value = float(value)
                
                line_dict[key] = value
        parsed_data.append(line_dict)
    return parsed_data

def replace_w2c(camera_dict, keycamera_dict):
    new_dict = camera_dict.copy()
    new_dict['position'] = keycamera_dict['position']
    new_dict['rotation'] = keycamera_dict['rotation']
    return new_dict

def get_forward_direction(camera_dict):
    return np.array(camera_dict['rotation'][2]).copy()

## Load cameras, keycameras

In [54]:
file_path = "./key_cameras_4"
parsed_data = parse_keycamera(file_path)
print(parsed_data)

for keycamera in parsed_data:
    print(convert_to_W2C(keycamera))

keycameras = [convert_to_W2C(keycamera) for keycamera in parsed_data]

[{'origin': [-0.60674, -0.077124, -3.563149], 'target': [-0.489422, -0.00529, -2.572657], 'up': [-0.928382, 0.362077, 0.083703], 'fovy': 51.702257, 'clip': [0.009, 1100.0]}, {'origin': [-0.189939, 0.178081, -0.044188], 'target': [-0.072621, 0.249914, 0.946304], 'up': [-0.928382, 0.362077, 0.083703], 'fovy': 51.702257, 'clip': [0.009, 1100.0]}, {'origin': [0.196958, 0.362517, 3.118752], 'target': [0.088023, 0.336833, 2.125035], 'up': [-0.927332, 0.362683, 0.092284], 'fovy': 51.702257, 'clip': [0.009, 1100.0]}, {'origin': [0.196958, 0.362517, 3.118752], 'target': [-0.158984, -0.568402, 3.200573], 'up': [-0.927332, 0.362682, 0.092283], 'fovy': 51.702257, 'clip': [0.009, 1100.0]}]
{'position': [-0.60674, -0.077124, -3.563149], 'rotation': [[-0.5118741856091824, 0.5841785259260074, 0.6298573393665056], [0.2661281372143025, 0.8049570237249027, -0.5303018051438176], [0.8167990160925646, 0.10382504419870613, 0.5675030638748614]]}
{'position': [-0.189939, 0.178081, -0.044188], 'rotation': [[-0.

In [5]:
store_base_forward_direction = base_forward_direction.copy()
store_base_forward_direction

NameError: name 'base_forward_direction' is not defined

### Helpers

In [6]:
def move_forward(start_dict, distance):
    new_dict = start_dict.copy()
    rot = np.array(new_dict['rotation'])
    pos = np.array(new_dict['position'])
    
    forward_direction = rot[:, 2]
    pos = pos + forward_direction * distance
    new_dict['position'] = pos.tolist()
    return new_dict

def rise_relative_to_camera(start_dict, distance):
    new_dict = start_dict.copy()
    rot = np.array(new_dict['rotation'])
    pos = np.array(new_dict['position'])
    
    up_direction = -rot[:, 0]
    pos = pos + up_direction * distance
    new_dict['position'] = pos.tolist()
    return new_dict

def rotate_relative_to_camera(start_dict, angle):
    new_dict = start_dict.copy()
    rot = np.array(new_dict['rotation'])
    pos = np.array(new_dict['position'])
    
    rot = rot @ np.array([[np.cos(angle), 0, np.sin(angle)], [0, 1, 0], [-np.sin(angle), 0, np.cos(angle)]])
    new_dict['rotation'] = rot.tolist()
    return new_dict

def point_camera_at(start_dict, point):
    new_dict = start_dict.copy()
    rot = np.array(new_dict['rotation'])
    pos = np.array(new_dict['position'])
    
    forward_direction = point - pos
    forward_direction = forward_direction / np.linalg.norm(forward_direction)

    up_direction = np.array([1, 0, 0])
    right_direction = np.cross(up_direction, forward_direction)
    right_direction = right_direction / np.linalg.norm(right_direction)

    up_direction = np.cross(forward_direction, right_direction)
    up_direction = up_direction / np.linalg.norm(up_direction)
    
    # import itertools
    # import random
    # order = random.sample(list(itertools.permutations([up_direction, right_direction, forward_direction], 3)), 1)
    # print(order)
    rot = np.array([up_direction, right_direction, forward_direction])
    new_dict['rotation'] = rot.tolist()
    return new_dict

def point_camera_at_origin(start_dict):
    return point_camera_at(start_dict, np.array([0, 0, 0]))

def place_camera_at(start_dict, point):
    new_dict = start_dict.copy()
    new_dict['position'] = point.tolist()
    return new_dict

# def project_camera_orientation_to_upright_plane(start_dict):
#     new_dict = start_dict.copy()
#     rot = np.array(new_dict['rotation'])
#     pos = np.array(new_dict['position'])
    
#     forward_direction = rot[:, 2]
#     forward_direction[1] = 0
#     forward_direction = forward_direction / np.linalg.norm(forward_direction)

#     up_direction = np.array([0, 1, 0])
#     right_direction = np.cross(up_direction, forward_direction)
#     right_direction = right_direction / np.linalg.norm(right_direction)

#     up_direction = np.cross(forward_direction, right_direction)
#     up_direction = up_direction / np.linalg.norm(up_direction)
    
#     rot = np.array([up_direction, right_direction, forward_direction])
#     new_dict['rotation'] = rot.tolist()
#     return new_dict

def rotate_camera(start_dict):
    new_dict = start_dict.copy()
    pos, rot = get_pos_rot(start_dict)
    
    up_vector = -rot[:, 0]
    up_plane = np.array([1, 0, 0])  # Assuming the up plane is the y-axis
    
    # Calculate the angle between the camera's up vector and the up plane
    angle = np.arccos(np.dot(up_vector, up_plane) / (np.linalg.norm(up_vector) * np.linalg.norm(up_plane)))
    
    # Rotate the camera by the calculated angle around the cross product of the up vector and the up plane
    rotation_axis = np.cross(up_vector, up_plane)
    rotation_matrix = np.array([[np.cos(angle) + rotation_axis[0]**2 * (1 - np.cos(angle)), rotation_axis[0] * rotation_axis[1] * (1 - np.cos(angle)) - rotation_axis[2] * np.sin(angle), rotation_axis[0] * rotation_axis[2] * (1 - np.cos(angle)) + rotation_axis[1] * np.sin(angle)],
                                [rotation_axis[1] * rotation_axis[0] * (1 - np.cos(angle)) + rotation_axis[2] * np.sin(angle), np.cos(angle) + rotation_axis[1]**2 * (1 - np.cos(angle)), rotation_axis[1] * rotation_axis[2] * (1 - np.cos(angle)) - rotation_axis[0] * np.sin(angle)],
                                [rotation_axis[2] * rotation_axis[0] * (1 - np.cos(angle)) - rotation_axis[1] * np.sin(angle), rotation_axis[2] * rotation_axis[1] * (1 - np.cos(angle)) + rotation_axis[0] * np.sin(angle), np.cos(angle) + rotation_axis[2]**2 * (1 - np.cos(angle))]])
    
    # Apply the rotation to the camera's position and up vector
    new_dict['position'] = (np.dot(rotation_matrix, pos)).tolist()
    new_dict['rotation'] = (rotation_matrix @ rot).tolist()
    # rot = np.dot(rotation_matrix, up_vector)
    # rot = rot / np.linalg.norm(rot)

    # new_dict['rotation'] = np.array([rot, np.cross(rot, np.array([0, 1, 0])), np.array([0, 1, 0])]).tolist()

    return new_dict

In [7]:


def rotate_about_forward_direction(start_dict, angle):
    new_dict = start_dict.copy()
    pos, rot = get_pos_rot(start_dict)

    # Define the rotation matrix for counterclockwise rotation about the forward direction
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle), 0], 
                                [np.sin(angle), np.cos(angle), 0],
                                [0, 0, 1]])

    # Apply the rotation to the camera's rotation matrix
    new_dict['rotation'] = (rotation_matrix @ rot).tolist()

    return new_dict


In [8]:
def interpolate(start_dict, end_dict, num_frames):
    start_pos, start_rot = get_pos_rot(start_dict)
    end_pos, end_rot = get_pos_rot(end_dict)

    pos_list = np.linspace(start_pos, end_pos, num_frames)
    rot_list = np.linspace(start_rot, end_rot, num_frames)

    dicts = []
    for i in range(num_frames):
        new_dict = start_dict.copy()
        new_dict['position'] = pos_list[i].tolist()
        new_dict['rotation'] = rot_list[i].tolist()
        dicts.append(new_dict)

    return dicts

interpolated = interpolate(keycameras[0], keycameras[1], 10)

for interpolate_dict in interpolated:
    save_list.append(rotate_about_forward_direction(replace_w2c(save_list[-1], interpolate_dict), np.pi/2))

IndexError: list index out of range

In [9]:
def flip_camera(start_dict):
    new_dict = start_dict.copy()
    pos, rot = get_pos_rot(start_dict)

    # Define the rotation matrix for counterclockwise rotation about the forward direction
    rotation_matrix = np.array([[1, 0, 0], 
                                [0, 1, 0],
                                [0, 0, -1]])

    # Apply the rotation to the camera's rotation matrix
    new_dict['rotation'] = (rot @ rotation_matrix).tolist()

    return new_dict

def camera_diff(camera1, camera2):
    pos1, rot1 = get_pos_rot(camera1)
    pos2, rot2 = get_pos_rot(camera2)

    pos_diff = pos1 - pos2
    rot_diff = rot1 - rot2

    print("Position difference: ", pos_diff)
    print("Rotation difference: ", rot_diff)

camera_diff(keycameras[2], flip_camera(keycameras[2]))

Position difference:  [0. 0. 0.]
Rotation difference:  [[ 0.          0.          0.18456717]
 [ 0.          0.          0.12665294]
 [ 0.          0.         -1.98743402]]


In [10]:
keycameras[2]

{'position': [0.196958, 0.362517, 3.118752],
 'rotation': [[-0.9273320764708398, 0.3626830013373253, 0.09228358732315443],
  [-0.3580340562910567, -0.9315585717873832, 0.06332647049396699],
  [-0.10893500118902552, -0.025684000280340846, -0.9937170108464213]]}

In [95]:
flip_camera(keycameras[2])

{'position': [0.196958, 0.362517, 3.118752],
 'rotation': [[-0.9273320764708398, 0.3626830013373253, -0.09228358732315443],
  [-0.3580340562910567, -0.9315585717873832, -0.06332647049396699],
  [-0.10893500118902552, -0.025684000280340846, 0.9937170108464213]]}

In [96]:
rotate_about_forward_direction(flip_camera(keycameras[2]), np.pi/2)

{'position': [0.196958, 0.362517, 3.118752],
 'rotation': [[0.35803405629105667, 0.9315585717873832, 0.06332647049396699],
  [-0.9273320764708398, 0.36268300133732523, -0.09228358732315443],
  [-0.10893500118902552, -0.025684000280340846, 0.9937170108464213]]}

In [97]:
rotate_about_forward_direction(keycameras[2], np.pi/2)

{'position': [0.196958, 0.362517, 3.118752],
 'rotation': [[0.35803405629105667, 0.9315585717873832, -0.06332647049396699],
  [-0.9273320764708398, 0.36268300133732523, 0.09228358732315443],
  [-0.10893500118902552, -0.025684000280340846, -0.9937170108464213]]}

In [49]:
# UP_VECTOR
np.array(rotate_about_forward_direction(keycameras[0], np.pi/2)['rotation'])[:, 1]
#array([ 0.93155857,  0.362683  , -0.025684  ])
#NEW: array([ 0.93149278,  0.36375419, -0.00202001])

array([ 0.93149278,  0.36375419, -0.00202001])

In [38]:
def rotate_about_up_direction(rot, angle):
    # Define the rotation matrix for counterclockwise rotation about the up direction
    rotation_matrix = np.array([
                                # [1, 0, 0],
                                # [0, np.cos(angle), -np.sin(angle)], 
                                # [0, np.sin(angle), np.cos(angle)],
                                [np.cos(angle), 0, -np.sin(angle)], 
                                [0, 1, 0],
                                [np.sin(angle), 0, np.cos(angle)],
                                ])

    # rotate on first row, is close to up direction
    # rotate on second row, is close to right direction
    # rotate on third row, is forward direction


    # Apply the rotation to the camera's rotation matrix
    new_rot = (rot @ rotation_matrix)
    # print(new_rot)
    return new_rot

def rotate_camera_dict_about_up_direction(camera_dict, angle):
    new_dict = camera_dict.copy()
    pos, rot = get_pos_rot(camera_dict)

    new_dict['rotation'] = rotate_about_up_direction(rot, angle).tolist()

    return new_dict

def dist_from_origin(camera_dict):
    pos, rot = get_pos_rot(camera_dict)
    return np.linalg.norm(pos)

## Generate Camera.json

In [55]:
save_list = []

start_dict = replace_w2c(cameras[185], keycameras[0])
base_forward_direction = get_forward_direction(start_dict)

first_append = start_dict.copy()
# first_append = flip_camera(first_append)
first_append = rotate_about_forward_direction(first_append, np.pi/2)
save_list.append(first_append)

print(save_list[-1])
camera_diff(keycameras[2], first_append)

INCREMENT_FORWARD = 0.05
INCREMENT_YAW = 2*np.pi / 80
START_TURNING_DIST = 1.5
accumulated_yaw = 0
has_turned = False
for i in range(200):
    last_dict = save_list[-1].copy()

    dist = dist_from_origin(last_dict)
    last_dict = move_forward(last_dict, INCREMENT_FORWARD)
    if dist < START_TURNING_DIST and abs(accumulated_yaw) <= np.pi / 2:
        last_dict = rotate_camera_dict_about_up_direction(last_dict, INCREMENT_YAW)
        accumulated_yaw += INCREMENT_YAW
        has_turned = True
    elif has_turned:
        break

    save_list.append(last_dict)

# write cameras[185] to a json file
with open('camera.json', 'w') as outfile:
    json.dump(save_list, outfile)

# for keycamera in keycameras:
#     save_list.append(rotate_about_forward_direction(replace_w2c(save_list[-1], keycamera), np.pi/2))

# for i in range(10):
#     save_list.append(rotate_camera(point_camera_at_origin(save_list[-1])))


{'id': 185, 'img_name': '298626350910', 'width': 1000, 'height': 752, 'position': [-0.60674, -0.077124, -3.563149], 'rotation': [[-0.26612813721430256, -0.8049570237249027, 0.5303018051438176], [-0.5118741856091824, 0.5841785259260074, 0.6298573393665056], [0.8167990160925646, 0.10382504419870613, 0.5675030638748614]], 'fy': 776.0253267729305, 'fx': 776.6956362332196}
Position difference:  [0.803698 0.439641 6.681901]
Rotation difference:  [[-0.66713145  1.1640575  -0.52173472]
 [ 0.15404599 -1.51568706 -0.56465888]
 [-0.84819213 -0.16160658 -1.5653386 ]]


In [54]:
rotate_about_forward_direction(start_dict, np.pi/2)

{'id': 185,
 'img_name': '298626350910',
 'width': 1000,
 'height': 752,
 'position': [0.196958, 0.362517, 3.118752],
 'rotation': [[0.35803405629105667, 0.9315585717873832, -0.06332647049396699],
  [-0.9273320764708398, 0.36268300133732523, 0.09228358732315443],
  [-0.10893500118902552, -0.025684000280340846, -0.9937170108464213]],
 'fy': 776.0253267729305,
 'fx': 776.6956362332196}